# ETL (Extract, Transform, Load)

Importamos las librerías necesarias:

In [1]:
import pandas as pd
import ast 
import re 

Lectura de "australian_user_reviews.json":

In [2]:
rows = []
with open ('data/australian_user_reviews.json', encoding='MacRoman') as f:
    for line in f.readlines():
        rows.append(ast.literal_eval(line))

df_reviews = pd.DataFrame(rows)


In [3]:
df_reviews.info(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25799 entries, 0 to 25798
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   user_id   25799 non-null  object
 1   user_url  25799 non-null  object
 2   reviews   25799 non-null  object
dtypes: object(3)
memory usage: 604.8+ KB


Guardar reviews en archivo csv:

In [4]:
df_reviews.to_csv('reviews.csv', index=False)

Lectura de "australian_users_items.json":

In [5]:
rows = []
with open ('data/australian_users_items.json', encoding='MacRoman') as f:
    for line in f.readlines():
        rows.append(ast.literal_eval(line))

df_items = pd.DataFrame(rows)

In [6]:
df_items.info(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88310 entries, 0 to 88309
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      88310 non-null  object
 1   items_count  88310 non-null  int64 
 2   steam_id     88310 non-null  object
 3   user_url     88310 non-null  object
 4   items        88310 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.4+ MB


Guardar items en archivo csv:

In [7]:
df_items.to_csv('items.csv', index=False)

Lectura de "output_steam_games.json":

In [8]:
df_games = pd.read_json('data/output_steam_games.json', lines=True)

In [9]:
df_games.info(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     24083 non-null  object 
 1   genres        28852 non-null  object 
 2   app_name      32133 non-null  object 
 3   title         30085 non-null  object 
 4   url           32135 non-null  object 
 5   release_date  30068 non-null  object 
 6   tags          31972 non-null  object 
 7   reviews_url   32133 non-null  object 
 8   specs         31465 non-null  object 
 9   price         30758 non-null  object 
 10  early_access  32135 non-null  float64
 11  id            32133 non-null  float64
 12  developer     28836 non-null  object 
dtypes: float64(2), object(11)
memory usage: 11.9+ MB


Guardar games en archivo CSV:

In [10]:
df_games.to_csv('games.csv', index=False)

**Depurarción de datos**

Leyendo archivo reviews:

In [11]:
df1= pd.read_csv('reviews.csv')

In [12]:
df1.head(3)

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."


In [13]:
# Hacer copia del primer dataframe

df1_reviews_copia = df1

Desanidando  columna review

In [14]:

data1 = df1_reviews_copia.loc[0, "reviews"]

# Crear lista utilizando ast.literal_eval
df1_lista = ast.literal_eval(data1)

# Conviertir a un DataFrame 
df1_reviews = pd.DataFrame(df1_lista)
df1_reviews.head(3)

,funny,posted,last_edited,item_id,helpful,recommend,review
0,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
2,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...


Funcion para convertir cada registro de formato str a lista:

In [15]:
def convertir_str_list(formato):
    list_registro = ast.literal_eval(formato) 
    return list_registro

In [16]:

df1_reviews_copia_2 = df1_reviews_copia.copy()
df1_reviews_copia_2['reviews'] = df1_reviews_copia_2['reviews'].apply(convertir_str_list) 

Desanidar columna "reviews":

In [17]:
list_complete_dict = []
for indice, fila in df1_reviews_copia_2.iterrows():       # recorre cada fila del dataframe
    user_id = fila['user_id']                           # guarda cada id de un usuario en una variable "user_id"
    user_url = fila['user_url']
    for dict_item_user in fila['reviews']:              # recorre cada fila de la columna "items"
        dict_item_user['user_id'] = user_id             # agrega una clave al diccionario que contiene el id del usuario
        dict_item_user['user_url'] = user_url
        list_complete_dict.append(dict_item_user)       # agrega cada dict a una lista general

df1_reviews_copia_3 = pd.DataFrame(list_complete_dict)    # crea un dataframe con la lista de dict
orden_columnas = ['user_id', 'user_url', "funny", "posted", "last_edited", 'item_id', 'helpful', 'recommend', 'review']
df1_reviews_copia_3 = df1_reviews_copia_3[orden_columnas]  

In [18]:
df1_reviews_copia_3.head(3) 

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...


Desanidar "items-csv"

In [19]:
df2= pd.read_csv('items.csv')

In [20]:
df2.head(3)

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."


In [21]:
df2_items_copia = df2

In [22]:
data2 = df2_items_copia.loc[0, "items"]

# Convertir la cadena a una lista
df2_lista = ast.literal_eval(data2)

# Convertir la lista en un DataFrame
df2_items= pd.DataFrame(df2_lista)
df2_items.head(10)

,item_id,item_name,playtime_forever,playtime_2weeks
0,10,Counter-Strike,6,0
1,20,Team Fortress Classic,0,0
2,30,Day of Defeat,7,0
3,40,Deathmatch Classic,0,0
4,50,Half-Life: Opposing Force,0,0
5,60,Ricochet,0,0
6,70,Half-Life,0,0
7,130,Half-Life: Blue Shift,0,0
8,300,Day of Defeat: Source,4733,0
9,240,Counter-Strike: Source,1853,0


In [23]:
df2_items_copia_2 = df2_items_copia.copy()
df2_items_copia_2['items'] = df2_items_copia_2['items'].apply(convertir_str_list)

In [24]:
list_complete_dict = []
for indice, fila in df2_items_copia_2.iterrows():       
    user_id = fila['user_id']                           
    items_count = fila['items_count']
    steam_id = fila['steam_id']
    user_url = fila['user_url']	
    for dict_item_user in fila['items']:            
        dict_item_user['user_id'] = user_id             
        dict_item_user['items_count'] = items_count
        dict_item_user['steam_id'] = steam_id
        dict_item_user['user_url'] = user_url
        list_complete_dict.append(dict_item_user)      

df2_items_copia_3 = pd.DataFrame(list_complete_dict)    
orden_columnas = ['user_id', "items_count", "steam_id",'user_url', "item_id", "item_name", "playtime_forever", "playtime_2weeks"]
df2_items_copia_3 = df2_items_copia_3[orden_columnas]   

In [25]:
df2_items_copia_3.head(3)

,user_id,items_count,steam_id,user_url,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,10,Counter-Strike,6,0
1,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,20,Team Fortress Classic,0,0
2,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,30,Day of Defeat,7,0


Lectura de games:

In [26]:
df3 = pd.read_csv('games.csv', low_memory=False)

**Verificar nulos de cada dataframe:**

In [27]:
# Pondre manos a la obra y revisare el tipo de datos de cada columna

def verificar_tipo_datos(df):
    
    mi_dict = {"nombre_campo": [], "tipo_datos": [], "no_nulos_%": [], "nulos_%": []} # Creo un diccionario para almacenar la información sobre las columnas del DataFrame.

    for columna in df.columns:  # Se recorren las columnas del DataFrame, y se calculan los porcentajes de valores no nulos y nulos para cada columna.
        porcentaje_no_nulos = (df[columna].count() / len(df)) * 100
        mi_dict["nombre_campo"].append(columna)
        mi_dict["tipo_datos"].append(df[columna].apply(type).unique())
        mi_dict["no_nulos_%"].append(round(porcentaje_no_nulos, 2))
        mi_dict["nulos_%"].append(round(100-porcentaje_no_nulos, 2))

    df_info = pd.DataFrame(mi_dict) # A partir de la información recopilada en el diccionario, se crea un nuevo DataFrame

    for columna in df.columns:
        print(columna, " (nulos) = ", df[columna].isnull().sum())
    
    print("\nfilas completamente nulas: ", df.isna().all(axis=1).sum())
    
    return df_info

In [28]:
# Verificando df1_reviews_copia
verificar = verificar_tipo_datos(df1_reviews_copia)

# imprimir la información en la salida
print(verificar)

user_id  (nulos) =  0
user_url  (nulos) =  0
reviews  (nulos) =  0

filas completamente nulas:  0
  nombre_campo       tipo_datos  no_nulos_%  nulos_%
0      user_id  [<class 'str'>]       100.0      0.0
1     user_url  [<class 'str'>]       100.0      0.0
2      reviews  [<class 'str'>]       100.0      0.0


In [29]:
# Verificando df2_itemns_copia2
verificar2 = verificar_tipo_datos(df2_items_copia_3)
print(verificar2)

user_id  (nulos) =  0
items_count  (nulos) =  0
steam_id  (nulos) =  0
user_url  (nulos) =  0
item_id  (nulos) =  0
item_name  (nulos) =  0
playtime_forever  (nulos) =  0
playtime_2weeks  (nulos) =  0

filas completamente nulas:  0
       nombre_campo       tipo_datos  no_nulos_%  nulos_%
0           user_id  [<class 'str'>]       100.0      0.0
1       items_count  [<class 'int'>]       100.0      0.0
2          steam_id  [<class 'int'>]       100.0      0.0
3          user_url  [<class 'str'>]       100.0      0.0
4           item_id  [<class 'str'>]       100.0      0.0
5         item_name  [<class 'str'>]       100.0      0.0
6  playtime_forever  [<class 'int'>]       100.0      0.0
7   playtime_2weeks  [<class 'int'>]       100.0      0.0


In [30]:
# Verificando 'df3=output_games.csv'
verificar3 = verificar_tipo_datos(df3)
print(verificar3)

publisher  (nulos) =  96381
genres  (nulos) =  91593
app_name  (nulos) =  88312
title  (nulos) =  90360
url  (nulos) =  88310
release_date  (nulos) =  90377
tags  (nulos) =  88473
reviews_url  (nulos) =  88312
specs  (nulos) =  88980
price  (nulos) =  89687
early_access  (nulos) =  88310
id  (nulos) =  88312
developer  (nulos) =  91609

filas completamente nulas:  88310
    nombre_campo                        tipo_datos  no_nulos_%  nulos_%
0      publisher  [<class 'float'>, <class 'str'>]       19.98    80.02
1         genres  [<class 'float'>, <class 'str'>]       23.95    76.05
2       app_name  [<class 'float'>, <class 'str'>]       26.68    73.32
3          title  [<class 'float'>, <class 'str'>]       24.98    75.02
4            url  [<class 'float'>, <class 'str'>]       26.68    73.32
5   release_date  [<class 'float'>, <class 'str'>]       24.96    75.04
6           tags  [<class 'float'>, <class 'str'>]       26.54    73.46
7    reviews_url  [<class 'float'>, <class 'str'>] 

Eliminar nulos de output_games

In [31]:
df3.dropna(how='all', inplace=True)  # Eliminar filas con todos los valores nulos
df3.reset_index(drop=True, inplace=True) 

In [32]:
# Eliminar algunas columnas que se consideraron no relevantes.
columnas_a_eliminar = ['reviews_url', 'url', 'early_access']
df3 = df3.drop(columnas_a_eliminar, axis=1)

In [33]:
df3 = df3.dropna()

In [34]:
nulos = df3.isnull().sum()

print("Valores nulos por columna:")
print(nulos)

Valores nulos por columna:
publisher       0
genres          0
app_name        0
title           0
release_date    0
tags            0
specs           0
price           0
id              0
developer       0
dtype: int64


**Exportar Dataframe parcialmente limpio :**

In [35]:
df1_reviews_copia_3.to_csv('reviews.csv', index=False)

In [36]:
df2_items_copia_3.to_csv('items.csv', index=False)

In [37]:
df3.to_csv('games.csv', index=False)

**Verificar y comprobar datos**

## reviews 

In [38]:
df_review= pd.read_csv('reviews.csv')

In [39]:
df_review.head(3)

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,NaN,"Posted November 5, 2011.",NaN,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,NaN,"Posted July 15, 2011.",NaN,22200,No ratings yet,True,It's unique and worth a playthrough.
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,NaN,"Posted April 21, 2011.",NaN,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...


In [40]:
#función ispección, para verifacar datos faltantes

def data_inspeccion(variables):
    print(f'Shape: {variables.shape}')
    print(f'\nDatos faltantes:')
    for columna in variables.columns:
        cantidad_nulos = df_review[columna].isnull().sum()
        print(f'{columna}: {cantidad_nulos} valores nulos')

data_inspeccion(df_review)

Shape: (59305, 9)

Datos faltantes:
user_id: 0 valores nulos
user_url: 0 valores nulos
funny: 51154 valores nulos
posted: 0 valores nulos
last_edited: 53165 valores nulos
item_id: 0 valores nulos
helpful: 0 valores nulos
recommend: 0 valores nulos
review: 30 valores nulos


In [41]:
#descartar nulos y comprobar de nuevo
df_review.dropna(inplace=True)
data_inspeccion(df_review)

Shape: (1160, 9)

Datos faltantes:
user_id: 0 valores nulos
user_url: 0 valores nulos
funny: 0 valores nulos
posted: 0 valores nulos
last_edited: 0 valores nulos
item_id: 0 valores nulos
helpful: 0 valores nulos
recommend: 0 valores nulos
review: 0 valores nulos


In [42]:
df_review.head(3)

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review
24,Wackky,http://steamcommunity.com/id/Wackky,1 person found this review funny,"Posted October 21, 2012.","Last edited November 25, 2013.",550,1 of 1 people (100%) found this review helpful,True,This game is fantastic if you are looking to D...
214,kevinator23,http://steamcommunity.com/id/kevinator23,1 person found this review funny,"Posted July 6, 2015.",Last edited March 8.,218620,2 of 5 people (40%) found this review helpful,True,"PayDay 2 on the difficulty scale -Normal - ""lo..."
239,Lord_Exploit,http://steamcommunity.com/id/Lord_Exploit,1 person found this review funny,"Posted March 13, 2015.","Last edited March 19, 2015.",311560,6 of 10 people (60%) found this review helpful,True,So after 30 odd hours i thought i'd write a re...


Eliminar columnas irrelevantes:

In [43]:

reviews_drop = ["user_url", "posted", "last_edited"]
df_review.drop(columns=reviews_drop, inplace=True)

In [44]:
df_review.head(3)

,user_id,funny,item_id,helpful,recommend,review
24,Wackky,1 person found this review funny,550,1 of 1 people (100%) found this review helpful,True,This game is fantastic if you are looking to D...
214,kevinator23,1 person found this review funny,218620,2 of 5 people (40%) found this review helpful,True,"PayDay 2 on the difficulty scale -Normal - ""lo..."
239,Lord_Exploit,1 person found this review funny,311560,6 of 10 people (60%) found this review helpful,True,So after 30 odd hours i thought i'd write a re...


In [45]:
# Ordenar el índice de manera ascendente
df_review = df_review.sort_index()

# Restablecer el índice para que los números estén en orden
df_review = df_review.reset_index(drop=True) # Esto evita que se agregue una columna adicional con el índice anterior.

df_review.head(3)

,user_id,funny,item_id,helpful,recommend,review
0,Wackky,1 person found this review funny,550,1 of 1 people (100%) found this review helpful,True,This game is fantastic if you are looking to D...
1,kevinator23,1 person found this review funny,218620,2 of 5 people (40%) found this review helpful,True,"PayDay 2 on the difficulty scale -Normal - ""lo..."
2,Lord_Exploit,1 person found this review funny,311560,6 of 10 people (60%) found this review helpful,True,So after 30 odd hours i thought i'd write a re...


Limpiar texto de columna review:

In [46]:
def clean_text(text):
    # Este codigo utiliza una expresión regular para eliminar todo lo que no sea texto y espacios en blanco
    cleaned_text = re.sub(r'[^a-zA-Z\s]', '', text)
    return cleaned_text

# Aplicó la función a la columna 'review'
df_review['review'] = df_review['review'].apply(clean_text)

# Items

In [47]:
df_items = pd.read_csv('items.csv')
df_items.head()

,user_id,items_count,steam_id,user_url,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,10,Counter-Strike,6,0
1,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,20,Team Fortress Classic,0,0
2,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,30,Day of Defeat,7,0
3,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,40,Deathmatch Classic,0,0
4,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,50,Half-Life: Opposing Force,0,0


In [48]:
df_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5153209 entries, 0 to 5153208
Data columns (total 8 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   user_id           object
 1   items_count       int64 
 2   steam_id          int64 
 3   user_url          object
 4   item_id           int64 
 5   item_name         object
 6   playtime_forever  int64 
 7   playtime_2weeks   int64 
dtypes: int64(5), object(3)
memory usage: 314.5+ MB


Eliminar columnas irrelevantes 

In [49]:
items_drop = ["steam_id","user_url"]
df_items.drop(columns=items_drop, inplace=True)

In [50]:
# verificar mis datos con el método duplicated y .any() si hay duplicados
duplicados = df_items.duplicated().any()
print(f"¿Hay duplicados en el DataFrame? {duplicados}")

¿Hay duplicados en el DataFrame? True


In [51]:
df_items = df_items.drop_duplicates()

In [52]:
df_items.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5094105 entries, 0 to 5153208
Data columns (total 6 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   user_id           object
 1   items_count       int64 
 2   item_id           int64 
 3   item_name         object
 4   playtime_forever  int64 
 5   playtime_2weeks   int64 
dtypes: int64(4), object(2)
memory usage: 272.1+ MB


Limpieza de texto

In [53]:
def clean_text(text):
    # Este codigo utiliza una expresión regular para eliminar todo lo que no sea texto y espacios en blanco
    cleaned_text = re.sub(r'[^a-zA-Z\s]', '', text)
    return cleaned_text

# Aplicó la función a la columna 'item_name'
df_items['item_name'] = df_items['item_name'].apply(clean_text)

# Games


In [54]:

df_games= pd.read_csv('games.csv')

verificar duplicados

In [55]:
duplicados = df_games[df_games.duplicated()]
duplicados

,publisher,genres,app_name,title,release_date,tags,specs,price,id,developer
10093,Bethesda Softworks,['Action'],Wolfenstein II: The New Colossus,Wolfenstein II: The New Colossus,2017-10-26,"['Action', 'FPS', 'Gore', 'Violent', 'Alternat...","['Single-player', 'Steam Achievements', 'Full ...",59.99,612880.0,Machine Games


In [56]:
df_games = df_games.drop_duplicates()

Limpieza de texto:

In [57]:
def clean_text(text):
    cleaned_text = re.sub(r'[^a-zA-Z\s]', '', text)
    return cleaned_text

# Columnas que voy a limpiar
columns_to_clean = ['genres', 'tags', 'specs', 'publisher', 'app_name', 'title', 'developer']

# Aplicó la función a las columnas especificadas
for column in columns_to_clean:
    df_games[column] = df_games[column].apply(clean_text)

In [58]:
df_games.head(3)


,publisher,genres,app_name,title,release_date,tags,specs,price,id,developer
0,Kotoshiro,Action Casual Indie Simulation Strategy,Lost Summoner Kitty,Lost Summoner Kitty,2018-01-04,Strategy Action Indie Casual Simulation,Singleplayer,4.99,761140.0,Kotoshiro
1,Making Fun Inc,Free to Play Indie RPG Strategy,Ironbound,Ironbound,2018-01-04,Free to Play Strategy Indie RPG Card Game Trad...,Singleplayer Multiplayer Online MultiPlayer Cr...,Free To Play,643980.0,Secret Level SRL
2,Poolianscom,Casual Free to Play Indie Simulation Sports,Real Pool D Poolians,Real Pool D Poolians,2017-07-24,Free to Play Simulation Sports Casual Indie Mu...,Singleplayer Multiplayer Online MultiPlayer In...,Free to Play,670290.0,Poolianscom


**Se exportan los dataframes parcialmente limpios**

In [59]:
df_items.to_csv('items.csv', index=False)

In [60]:
df_review.to_csv('reviews.csv', index=False)

In [61]:
df_games.to_csv('games.csv', index=False)

**Feature Engineering: Columna 'sentiment_analysis'**

In [62]:
# TextBlob es una biblioteca para procesar datos textuales.
# Proporciona una API  para realizar tareas comunes de procesamiento del lenguaje natural (NLP).
from textblob import TextBlob

In [63]:
def analisis(review):
    analisis = TextBlob(str(review))
    # Clasificar la polaridad de la reseña
    if analisis.sentiment.polarity > 0:
        return 2  # Positivo
    elif analisis.sentiment.polarity < 0:
        return 0  # Malo
    else:
        return 1  # Neutral

# Aplicar el análisis de sentimiento a la columna 'review' y crear 'sentiment_analysis'
df_review['sentiment_analysis'] = df_review['review'].apply(analisis)

# Reemplazar la columna 'review' con 'sentiment_analysis'
df_review.drop(columns=['review'], inplace=True)

# Rellenar con 1 si la reseña está ausente
df_review['sentiment_analysis'].fillna(1, inplace=True)

In [64]:
df_review.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1160 entries, 0 to 1159
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   user_id             1160 non-null   object
 1   funny               1160 non-null   object
 2   item_id             1160 non-null   int64 
 3   helpful             1160 non-null   object
 4   recommend           1160 non-null   bool  
 5   sentiment_analysis  1160 non-null   int64 
dtypes: bool(1), int64(2), object(3)
memory usage: 46.6+ KB


## Ahora que todos los datos están depurados, ordenados y listos para el analisis, se exportan:

In [65]:
df_review.to_csv('reviews.csv', index=False)

In [66]:
df_items.to_csv('items.csv', index=False)

In [ ]:
df_games.to_csv('games.csv', index=False)